In [43]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
from peft import (
    PeftModel,
)
from typing import Optional, Union, List, Dict, Any
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
import re

# import re
# from transformers import AutoTokenizer
# from tokenizers.decoders import ByteLevel

# bytelevel_decoder = ByteLevel()

# def get_vocab_from_bytelevel_decoder(bytelevel_tokenizer):
#     subwords = list(map(lambda subword: bytelevel_decoder.decode([subword]).strip(), bytelevel_tokenizer.get_vocab().keys()))
#     return subwords

# bytelevel_subwords = get_vocab_from_bytelevel_decoder(tokenizer)
# # len(bytelevel_subwords), bytelevel_subwords[:10]

# bytelevel_subwords_th = list(filter(lambda x: re.search(r'[ก-๙)]', x), bytelevel_subwords))
# bytelevel_subwords_th[:10]

## Load Pretrained and PEFT Model

In [6]:
tokenizer = AutoTokenizer.from_pretrained('facebook/xglm-7.5B')

In [7]:
pretrained_model = AutoModelForCausalLM.from_pretrained(
    'facebook/xglm-7.5B',
#     device_map="auto",
#     torch_dtype="auto",
)

pretrained_model.eval()
pretrained_model.get_memory_footprint()

30004674560

In [8]:
model = PeftModel.from_pretrained(pretrained_model, 
                                  'pythainlp/adapter-wangchanglm-7.5B-lora-multiworld')
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!
model.eval()
model.get_memory_footprint()

Downloading:   0%|          | 0.00/393 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

31212634112

## Get Tokens to Exclude from Beginning of Generation

In [44]:
exclude_pattern = re.compile(r'[^ก-๙]+') #|[^0-9a-zA-Z]+
def is_exclude(text):
   return bool(exclude_pattern.search(text))

df = pd.DataFrame(tokenizer.vocab.items(), columns=['text', 'idx'])
df['is_exclude'] = df.text.map(is_exclude)
exclude_ids = df[df.is_exclude==True].idx.tolist()
[tokenizer.decode(i) for i in exclude_ids[:10]]

['(2018)',
 'йд',
 'краю',
 'zaposlen',
 'ಕಥೆ',
 'അതി',
 'коју',
 'پولی',
 'krásne',
 'eenth']

## Context-reminded Logit Processor

In [45]:
from transformers import LogitsProcessor

class FocusContextProcessor(LogitsProcessor):
    def __init__(self,
        context_tokens: List[int],
        vocab_size: int,
        scaling_factor: float=1.5,
        device='cpu'):
        self.context_tokens = context_tokens
        self.vocab_size = vocab_size
        self.scaling_factor = scaling_factor
        self.context_mask = torch.tensor([self.scaling_factor if \
            t in self.context_tokens else 1 for t in range(self.vocab_size)]).to(device)

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        scores_pos = scores.clamp(min=0) * self.context_mask
        scores_neg = scores.clamp(max=0) / self.context_mask
        return scores_pos + scores_neg

## Beam Search

In [29]:
context = '''
แฟนบอลลิเวอร์พูล ถูกใจสนับสนุนอยากให้สโมสรคว้าตัว ไรอัน กราเวนเบิร์ช มิดฟิลด์ดาวรุ่งชาวดัตช์ของ บาเยิร์น มิวนิค มาร่วมทีมซัมเมอร์นี้ หลังปรากฎภาพแข้งวัย 20 ปี เคยสวมเสื้อแข่งหงส์แดงมาก่อนในอดีต
ลิเวอร์พูล ถูกพูดถึงอย่างมากกับความเคลื่อนไหวในตลาดซัมเมอร์นี้ หลังสโมสรถอนตัวยอมรับว่าไม่มีเงินมากพอไล่ล่าตัว จู๊ด เบลลิ่งแฮม วันเดอร์คิดของโบรุสเซีย ดอร์ทมุนด์ แต่จะมองหาเป้าหมายอื่นที่มีความเหมาะสมแทน
กราเวนเบิร์ช เป็นหนึ่งในตัวเลือกที่ลิเวอร์พูลให้ความสนใจ เนื่องจากดาวรุ่งวัย 20 ปี กำลังมีปัญหาแทบไม่ค่อยได้รับโอกาสจากบาเยิร์น มิวนิค นับตั้งแต่ย้ายมาจากอาแย๊กซ์ ค่าตัว 21 ล้านปอนด์ เมื่อซัมเมอร์ที่แล้ว
โดย The Times เปิดเผยว่า ตัวแทนของลิเวอร์พูล ได้เดินทางไปพบกับคุณพ่อของกราเวนเบิร์ช ถึงเนเธอร์แลนด์ เมื่อวันอังคารที่ผ่านมา ซึ่งคาดว่า เสือใต้พร้อมที่จะปล่อยตัวนักเตะให้ย้ายทีมในราคาประมาณ 25 ล้านปอนด์
'''

input_text = f"<context>: {context} <human>: สรุปสั้นๆ <bot>: "
# input_text = f"<human>: เล่าเรื่องเกี่ยวกับมังกรหน่อย <bot>: "
batch = tokenizer(input_text, return_tensors='pt')

In [ ]:
#remind the models of the tokens in context
context_tokens = tokenizer(context, add_special_tokens=False)['input_ids']
logits_processor = FocusContextProcessor(context_tokens, 
                                         model.config.vocab_size, 
                                         scaling_factor = 1.5)

In [46]:
output_tokens = pretrained_model.generate(**batch, 
                               no_repeat_ngram_size=2,
                               num_beams=5,
                               min_length=len(batch['input_ids'])+128,
                               max_new_tokens=512,
                               begin_suppress_tokens = exclude_ids,
                               logits_processor=[logits_processor],
                              )

print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

<context>: แฟนบอลลิเวอร์พูล ถูกใจสนับสนุนอยากให้สโมสรคว้าตัว ไรอัน กราเวนเบิร์ช มิดฟิลด์ดาวรุ่งชาวดัตช์ของ บาเยิร์น มิวนิค มาร่วมทีมซัมเมอร์นี้ หลังปรากฎภาพแข้งวัย 20 ปี เคยสวมเสื้อแข่งหงส์แดงมาก่อนในอดีต ลิเวอร์พูล ถูกพูดถึงอย่างมากกับความเคลื่อนไหวในตลาดซัมเมอร์นี้ หลังสโมสรถอนตัวยอมรับว่าไม่มีเงินมากพอไล่ล่าตัว จู๊ด เบลลิ่งแฮม วันเดอร์คิดของโบรุสเซีย ดอร์ทมุนด์ แต่จะมองหาเป้าหมายอื่นที่มีความเหมาะสมแทน กราเวนเบิร์ช เป็นหนึ่งในตัวเลือกที่ลิเวอร์พูลให้ความสนใจ เนื่องจากดาวรุ่งวัย 20 ปี กําลังมีปัญหาแทบไม่ค่อยได้รับโอกาสจากบาเยิร์น มิวนิค นับตั้งแต่ย้ายมาจากอาแย๊กซ์ ค่าตัว 21 ล้านปอนด์ เมื่อซัมเมอร์ที่แล้ว โดย The Times เปิดเผยว่า ตัวแทนของลิเวอร์พูล ได้เดินทางไปพบกับคุณพ่อของกราเวนเบิร์ช ถึงเนเธอร์แลนด์ เมื่อวันอังคารที่ผ่านมา ซึ่งคาดว่า เสือใต้พร้อมที่จะปล่อยตัวนักเตะให้ย้ายทีมในราคาประมาณ 25 ล้านปอนด์ <human>: ทําไมลิเวอร์พูลซื้อจู๊ด เบลลิ่งแฮมไม่ได้? <bot>: สโมสรลิเวอร์พูลถูกวิจารณ์อย่างหนักจากตลาดซื้อขายนักเตะในฤดูกาลนี้ เนื่องจากสโมสรไม่สามารถหาเงินมาจ่ายค่าตัวของนักเตะที่พวกเขาต